# Feature Engineering en SQL

A continuación, veremos cómo calcular diferentes variables para el feature engineering utilizando SQL.


## 1. Configuraciones generales.

In [ ]:
%pip install duckdb
%pip install jupysql
%pip install duckdb-engine

In [ ]:
# Librerias.
import duckdb
import pandas as pd
    
# Configuración de jupysql.
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Conexión a DuckDB.
%sql duckdb:///:memory:

In [3]:
# Datasets.
dataset_path = 'C:/Users/c678456/Desktop/Ian/Maestría/Especializacion/2do_cuatrimestre/DMEyF/datasets/'
dataset_file = 'competencia_01.csv'

## 2. Manejo de datos.

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

In [ ]:
%%sql
select
    Master_Fvencimiento
    , Visa_Fvencimiento
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
from competencia_01 limit 10

Lo siguiente es querer operar dos variables, como por ejemplo sumarla. Esto es sencillo


In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 limit 10

Pero un DS de a de veras mirará los datos y se encontrará con un campo que es null cuando se lo suma a otro dará null.

In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , Master_msaldototal + Visa_msaldototal as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10

Esto no siempre es deseable y puede ser fácilmente evitable

In [ ]:
%%sql
select
    Master_msaldototal
    , Visa_msaldototal
    , ifnull(Master_msaldototal, 0) + ifnull(Visa_msaldototal, 0) as tc_saldo_total
from competencia_01 limit 10

In [ ]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);


In [ ]:
%%sql
select distinct
    Master_msaldototal
    , Visa_msaldototal
    , suma_sin_null(Master_msaldototal, Visa_msaldototal) as tc_saldo_total
from competencia_01 where Master_msaldototal is null limit 10


TAREA: Escriba una macro para hacer un ratio de dos variables que sea seguro, donde no solo hay campos con null, también esta el problema de la división por cero. Como es costumbre comparta su solución por este canal. Lea https://duckdb.org/docs/sql/functions/numeric.html para referencias de funciones que puede usar.

---

"Claro!" me dirá, mientras lee esto con un mate en la mano, "para cosas fáciles usar SQL alcanza, pero para algo más complicado como crear campos contra el data drifting es difícil".... elija su medicina:

In [ ]:
%%sql
select
    foto_mes
    , numero_de_cliente
    , cliente_antiguedad
    , row_number() over (partition by numero_de_cliente order by foto_mes) as cliente_antiguedad_2 # Enumero los meses por cliente
    , percent_rank() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_3 # Posición de los clientes por mes según su antigüedad, entre 0 y 1.
    , cume_dist() over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_4 # Distribución acumulativa de los valores de antigüedad dentro de cada mes.
    , ntile(4) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_5 # Corto la poblacion en 4 grupos, y le asigno una a cada cliente, según su posición de antigüedad.
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6 # Corto la poblacion en 10, y le asigno una a cada cliente, según su posición de antigüedad.
from competencia_01
order by numero_de_cliente, cliente_antiguedad


Qué paso? use las hermosas funciones analíticas de SQL. Al campo cliente_antiguedad (que no sufre de data drifting, solo esta para dar el ejemplo) para cada período (partition by foto_mes) la ordeno (order by cliente_antiguedad) y luego calculo las métricas de orden que pueden encontrar acá https://duckdb.org/docs/sql/window_functions.html#general-purpose-window-functions.

Seguiremos usando las funciones analíticas de SQL, esta vez para calcular features que utilizan valores del pasado.

Qué pasa si quiero agregar un feature que muestre el valor del periodo anterior?


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter # Me traigo el valor del mes anterior para el cliente en cuestión.
from competencia_01
limit 10


Podemos calcular el delta (diferencia) entre el valor pasado y el presente, para uno o varios meses


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , ctrx_quarter - lag_1_ctrx_quarter as delta_1_ctrx_quarter
  , ctrx_quarter - lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
from competencia_01
limit 10


Si necesitamos ya no solo traer un valor del pasado, sino una secuencia de valores, por ejemplo para calcular la media móvil con los últimos 3 meses anteriores? se puede hacer fácilmente


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , lag(ctrx_quarter, 1) over (partition by numero_de_cliente order by foto_mes) as lag_1_ctrx_quarter
  , lag(ctrx_quarter, 2) over (partition by numero_de_cliente order by foto_mes) as lag_2_ctrx_quarter
  , lag(ctrx_quarter, 3) over (partition by numero_de_cliente order by foto_mes) as lag_3_ctrx_quarter
  , avg(ctrx_quarter) over (partition by numero_de_cliente
                            order by foto_mes
                            rows between 3 preceding and current row) as avg_3_ctrx_quarter
from competencia_01
order by numero_de_cliente, foto_mes desc
limit 10


Si embargo puede resultar incómodo escribir constantemente el over partition sobre todo si se buscan aplicar muchas veces para distintas funciones. Para reducir el código se puede usar la siguiente sintaxis



In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  , avg(ctrx_quarter) over ventana_3 as ctrx_quarter_media_3
  , max(ctrx_quarter) over ventana_3 as ctrx_quarter_max_3
  , min(ctrx_quarter) over ventana_3 as ctrx_quarter_min_3
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


Para saber más que funciones tenemos disponibles, recomiendo ver los siguientes links:

https://duckdb.org/docs/archive/0.8.1/sql/window_functions
https://duckdb.org/docs/archive/0.8.1/sql/aggregates
Un caso más, que ni me voy a molestar en explicar que significa...


In [ ]:
%%sql
select
  numero_de_cliente
  , foto_mes
  , ctrx_quarter
  ,regr_slope(ctrx_quarter, cliente_antiguedad) over ventana_3 as ctrx_quarter_slope_3 # La pendiente de la regresión lineal.
from competencia_01
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)
limit 10


... Alguno dirá "tenemos que escribir todo esto a mano? Son muchas variables!". Bueno no, use los conocimientos de programación para que la computadora trabaje para usted. Si tenemos una lista de campos


In [28]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad']


Podemos hacer un script muy sencillo que nos genere el texto que hay que poner en una query para generar esas variables


In [ ]:
campos = ['active_quarter', 'cliente_vip', 'internet', 'cliente_edad', 'cliente_antiguedad', 'mrentabilidad']
nuevos_features = ""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as ctrx_{campo}_slope_3"
print(nuevos_features)





Con la salida de esa celda, arme la query agregando las nuevas líneas y la ejecuta.

Lo que acabamos de hacer de manera muy simple es como "funcionan" sistemas como **dbt** que están tan de moda en el mundo de los datos.

La última reflexión, la creación de nuevas features es un proceso computacionalmente rápido pero intenso. Si ejecutó lo anterior pudo haber visto que en poco minutos tenía sus nuevas variables. Pero, también pudo haberle fallado por temas de recursos. Miles de variables necesitan los recursos adecuados. Use la nube, una máquina grande, al menos que sepa bien como optimizar las queries.


Y a no olvidarse guardar las nueva tabla

In [ ]:
%%sql
COPY competencia_01 TO '{dataset_path}competencia_01_fe.csv' (FORMAT CSV, HEADER TRUE);

## 3. Desarrollo propio.

#### A. Configuraciones básicas.

In [1]:
# Librerias.
import duckdb
import pandas as pd
    
# Configuración de jupysql.
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

# Conexión a DuckDB.
%sql duckdb:///:memory:
    
# Datasets.
dataset_path = 'C:/Users/c678456/Desktop/Ian/Maestría/Especializacion/2do_cuatrimestre/DMEyF/datasets/'
dataset_file = 'competencia_01.csv'

#### B. Lectura.

In [ ]:
%%sql
create or replace table competencia_01 as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

#### C. Feature Engineering - Creando relaciones entre variables.

In [ ]:
%%sql
CREATE OR REPLACE MACRO suma_sin_null(a, b) AS ifnull(a, 0) + ifnull(b, 0);

In [ ]:
%%sql
CREATE OR REPLACE MACRO division_segura(a, b) AS 
    CASE 
        WHEN ifnull(b, 0) = 0 THEN NULL 
        ELSE ifnull(a, 0) / ifnull(b, 1) 
    END;

In [5]:
# Sumo Métricas de VISA + MASTERCARD para tener un comportamiento total.

In [ ]:
%%sql
create or replace table competencia_01_sumas as
select *,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as tc_consumo_total
    --, suma_sin_null(mtarjeta_visa_debitos_automaticos, mtarjeta_master_debitos_automaticos) as tc_debitosautomaticos_total
    , suma_sin_null(Master_mfinanciacion_limite,Visa_mfinanciacion_limite) as tc_financiacionlimite_total
    , suma_sin_null(Master_msaldopesos,Visa_msaldopesos) as tc_saldopesos_total
    , suma_sin_null(Master_msaldodolares,Visa_msaldodolares) as tc_saldodolares_total
    , suma_sin_null(Master_mconsumospesos,Visa_mconsumospesos) as tc_consumopesos_total
    , suma_sin_null(Master_mconsumosdolares,Visa_mconsumosdolares) as tc_consumodolares_total
    , suma_sin_null(Master_mlimitecompra,Visa_mlimitecompra) as tc_limitecompra_total
    , suma_sin_null(Master_madelantopesos,Visa_madelantopesos) as tc_adelantopesos_total
    , suma_sin_null(Master_madelantodolares,Visa_madelantodolares) as tc_adelantodolares_total
    , suma_sin_null(tc_adelantopesos_total,tc_adelantodolares_total) as tc_adelanto_total
    , suma_sin_null(Master_mpagado,Visa_mpagado) as tc_pagado_total
    , suma_sin_null(Master_mpagospesos,Visa_mpagospesos) as tc_pagadopesos_total
    , suma_sin_null(Master_mpagosdolares,Visa_mpagosdolares) as tc_pagadodolares_total
    , suma_sin_null(Master_msaldototal,Visa_msaldototal) as tc_saldototal_total
    , suma_sin_null(Master_mconsumototal,Visa_mconsumototal) as tc_consumototal_total
    , suma_sin_null(Master_cconsumos,Visa_cconsumos) as tc_cconsumos_total
    , suma_sin_null(Master_delinquency,Visa_delinquency) as tc_morosidad_total
from competencia_01

In [ ]:
%%sql
create or replace table competencia_01_sumas as
select *,
    suma_sin_null(mtarjeta_visa_consumo, mtarjeta_master_consumo) as tc_consumo_total
    --, suma_sin_null(mtarjeta_visa_debitos_automaticos, mtarjeta_master_debitos_automaticos) as tc_debitosautomaticos_total
    , suma_sin_null(Master_mfinanciacion_limite,Visa_mfinanciacion_limite) as tc_financiacionlimite_total
    , suma_sin_null(Master_msaldopesos,Visa_msaldopesos) as tc_saldopesos_total
    , suma_sin_null(Master_msaldodolares,Visa_msaldodolares) as tc_saldodolares_total
    , suma_sin_null(Master_mconsumospesos,Visa_mconsumospesos) as tc_consumopesos_total
    , suma_sin_null(Master_mconsumosdolares,Visa_mconsumosdolares) as tc_consumodolares_total
    , suma_sin_null(Master_mlimitecompra,Visa_mlimitecompra) as tc_limitecompra_total
    , suma_sin_null(Master_madelantopesos,Visa_madelantopesos) as tc_adelantopesos_total
    , suma_sin_null(Master_madelantodolares,Visa_madelantodolares) as tc_adelantodolares_total
    , suma_sin_null(tc_adelantopesos_total,tc_adelantodolares_total) as tc_adelanto_total
    , suma_sin_null(Master_mpagado,Visa_mpagado) as tc_pagado_total
    , suma_sin_null(Master_mpagospesos,Visa_mpagospesos) as tc_pagadopesos_total
    , suma_sin_null(Master_mpagosdolares,Visa_mpagosdolares) as tc_pagadodolares_total
    , suma_sin_null(Master_msaldototal,Visa_msaldototal) as tc_saldototal_total
    , suma_sin_null(Master_mconsumototal,Visa_mconsumototal) as tc_consumototal_total
    , suma_sin_null(Master_cconsumos,Visa_cconsumos) as tc_cconsumos_total
    , suma_sin_null(Master_delinquency,Visa_delinquency) as tc_morosidad_total
from competencia_01


In [ ]:
%%sql
create or replace table competencia_01_sumas as
select
    *
    , greatest(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_mayor
    , least(Master_Fvencimiento, Visa_Fvencimiento) as tc_fvencimiento_menor
    , greatest(Master_fechaalta, Visa_fechaalta) as tc_fechaalta_mayor
    , least(Master_fechaalta, Visa_fechaalta) as tc_fechalta_menor
    , greatest(Master_Finiciomora,Visa_Finiciomora) as tc_fechamora_mayor
    , least(Master_Finiciomora,Visa_Finiciomora) as tc_fechamora_menor
    , greatest(Master_fultimo_cierre,Visa_fultimo_cierre) as tc_fechacierre_mayor
    , least(Master_fultimo_cierre,Visa_fultimo_cierre) as tc_fechacierre_menor
from competencia_01_sumas

In [9]:
# Sumo Métricas de PESOS + DOLARES y PAYROLLS para tener un comportamiento total (mas allá de las incluidas previamente).

In [ ]:
%%sql
create or replace table competencia_01_sumas as
select *
    , suma_sin_null(mplazo_fijo_dolares, mplazo_fijo_pesos) as m_plazofijo_total
    , suma_sin_null(minversion1_dolares, minversion1_pesos) as m_inversion1_total
    , suma_sin_null(mpayroll, mpayroll2) as m_payroll_total
    , suma_sin_null(cpayroll_trx, cpayroll2_trx) as c_payroll_total
    , suma_sin_null(suma_sin_null(suma_sin_null(cseguro_vida, cseguro_auto), cseguro_vivienda), cseguro_accidentes_personales) as c_seguros_total
from competencia_01_sumas

In [11]:
# Calculo Métricas de posición ante antigüedad, fechas de altas, etc.

In [ ]:
%%sql
create or replace table competencia_01_sumas as
select
    *
    , ntile(10) over (partition by foto_mes order by cliente_antiguedad) as cliente_antiguedad_6 # Corto la poblacion en 10, y le asigno una a cada cliente, según su posición de antigüedad.
from competencia_01_sumas
order by numero_de_cliente, cliente_antiguedad

In [ ]:
%%sql
create or replace table competencia_01_sumas as
select
    *
    ,ntile(10) over (partition by foto_mes order by tc_fechaalta_mayor) as antiguedad_tarjetas
from competencia_01_sumas
order by numero_de_cliente, tc_fechaalta_mayor

In [14]:
# Calculo proporciones (Ej: Cantidades promedio, Visa/Mastercard, etc).

In [ ]:
%%sql
create or replace table competencia_01_sumas as
select *
    , division_segura(m_plazofijo_total, cplazo_fijo) as m_promedio_plazofijo_total
    , division_segura(m_inversion1_total, cinversion1) as m_promedio_inversion_total
    , division_segura(mcaja_ahorro, ccaja_ahorro) as m_promedio_caja_ahorro
    , division_segura(mtarjeta_visa_consumo, ctarjeta_visa_transacciones) as m_promedio_tarjeta_visa_consumo_por_transaccion
    , division_segura(mtarjeta_master_consumo, ctarjeta_master_transacciones) as m_promedio_tarjeta_master_consumo_por_transaccion
    , division_segura(mprestamos_personales, cprestamos_personales) as m_promedio_prestamos_personales
    , division_segura(mprestamos_prendarios, cprestamos_prendarios) as m_promedio_prestamos_prendarios
    , division_segura(mprestamos_hipotecarios, cprestamos_hipotecarios) as m_promedio_prestamos_hipotecarios
    , division_segura(minversion2, cinversion2) as m_promedio_inversion2
    , division_segura(mpagodeservicios, cpagodeservicios) as m_promedio_pagodeservicios
    , division_segura(mpagomiscuentas, cpagomiscuentas) as m_promedio_pagomiscuentas
    , division_segura(mcajeros_propios_descuentos, ccajeros_propios_descuentos) as m_promedio_cajeros_propios_descuentos
    , division_segura(mtarjeta_visa_descuentos, ctarjeta_visa_descuentos) as m_promedio_tarjeta_visa_descuentos
    , division_segura(mtarjeta_master_descuentos, ctarjeta_master_descuentos) as m_promedio_tarjeta_master_descuentos
    , division_segura(mcomisiones_mantenimiento, ccomisiones_mantenimiento) as m_promedio_comisiones_mantenimiento
    , division_segura(mcomisiones_otras, ccomisiones_otras) as m_promedio_comisiones_otras
    , division_segura(mforex_buy, cforex_buy) as m_promedio_forex_buy
    , division_segura(mforex_sell, cforex_sell) as m_promedio_forex_sell
    , division_segura(mtransferencias_recibidas, ctransferencias_recibidas) as m_promedio_transferencias_recibidas
    , division_segura(mtransferencias_emitidas, ctransferencias_emitidas) as m_promedio_transferencias_emitidas
    , division_segura(mextraccion_autoservicio, cextraccion_autoservicio) as m_promedio_extraccion_autoservicio
    , division_segura(mcheques_depositados, ccheques_depositados) as m_promedio_cheques_depositados
    , division_segura(mcheques_emitidos, ccheques_emitidos) as m_promedio_cheques_emitidos
    , division_segura(mcheques_depositados_rechazados, ccheques_depositados_rechazados) as m_promedio_cheques_depositados_rechazados
    , division_segura(mcheques_emitidos_rechazados, ccheques_emitidos_rechazados) as m_promedio_cheques_emitidos_rechazados
    , division_segura(matm, catm_trx) as m_promedio_atm
    , division_segura(matm_other, catm_trx_other) as m_promedio_atm_other
    , division_segura(Master_msaldototal,Master_mfinanciacion_limite) as proporcion_financiacion_master_cubierto
    , division_segura(Visa_msaldototal,Master_mlimitecompra) as proporcion_limite_master_cubierto
    , division_segura(Master_msaldototal,Visa_mfinanciacion_limite) as proporcion_financiacion_visa_cubierto
    , division_segura(Visa_msaldototal,Visa_mlimitecompra) as proporcion_limite_visa_cubierto
    , division_segura(tc_saldototal_total,tc_financiacionlimite_total) as proporcion_financiacion_total_cubierto
    , division_segura(tc_saldototal_total,tc_limitecompra_total) as proporcion_limite_total_cubierto
    , division_segura(tc_saldopesos_total,tc_saldototal_total) as tc_proporcion_saldo_pesos
    , division_segura(tc_saldodolares_total,tc_saldototal_total) as tc_proporcion_saldo_dolares
    , division_segura(tc_consumopesos_total,tc_consumototal_total) as tc_proporcion_consumo_pesos
    , division_segura(tc_consumodolares_total,tc_consumototal_total) as tc_proporcion_consumo_dolares
    , division_segura(tc_pagadopesos_total,tc_pagado_total) as tc_proporcion_pago_pesos
    , division_segura(tc_pagadodolares_total,tc_pagado_total) as tc_proporcion_pago_dolares
    , division_segura(tc_adelantopesos_total,tc_adelanto_total) as tc_proporcion_adelanto_pesos
    , division_segura(tc_adelantodolares_total,tc_adelanto_total) as tc_proporcion_adelanto_dolares
from competencia_01_sumas

#### D. Feature Engineering - Data Drifting.

In [16]:
campos_iniciales = [
    "mrentabilidad",
    "mrentabilidad_annual",
    "mcomisiones",
    "mactivos_margen",
    "mpasivos_margen",
    "cproductos",
    "mcuenta_corriente",
    "ccaja_ahorro",
    "mcaja_ahorro",
    "mcaja_ahorro_adicional",
    "mcaja_ahorro_dolares",
    "mcuentas_saldo",
    "ctarjeta_debito",
    "ctarjeta_debito_transacciones",
    "mautoservicio",
    "ctarjeta_visa_transacciones",
    "mtarjeta_visa_consumo",
    "ctarjeta_master_transacciones",
    "mtarjeta_master_consumo",
    "cprestamos_personales",
    "mprestamos_personales",
    "cprestamos_prendarios",
    "mprestamos_prendarios",
    "cprestamos_hipotecarios",
    "mprestamos_hipotecarios",
    "cplazo_fijo",
    "mplazo_fijo_dolares",
    "mplazo_fijo_pesos",
    "cinversion1",
    "minversion1_pesos",
    "minversion1_dolares",
    "cinversion2",
    "minversion2",
    "mpayroll",
    "mpayroll2",
    "ccuenta_debitos_automaticos",
    "mcuenta_debitos_automaticos",
    "ctarjeta_visa_debitos_automaticos",
    #"mtarjeta_visa_debitos_automaticos",
    #"ctarjeta_master_debitos_automaticos",
    "mttarjeta_master_debitos_automaticos",
    "cpagodeservicios",
    "mpagodeservicios",
    "cpagomiscuentas",
    "mpagomiscuentas",
    "ccajeros_propios_descuentos",
    "mcajeros_propios_descuentos",
    "ctarjeta_visa_descuentos",
    "mtarjeta_visa_descuentos",
    "ctarjeta_master_descuentos",
    "mtarjeta_master_descuentos",
    "ccomisiones_mantenimiento",
    "mcomisiones_mantenimiento",
    "ccomisiones_otras",
    "mcomisiones_otras",
    "cforex",
    "cforex_buy",
    "mforex_buy",
    "cforex_sell",
    "mforex_sell",
    "ctransferencias_recibidas",
    "mtransferencias_recibidas",
    "ctransferencias_emitidas",
    "mtransferencias_emitidas",
    "cextraccion_autoservicio",
    "mextraccion_autoservicio",
    "ccheques_depositados",
    "mcheques_depositados",
    "ccheques_emitidos",
    "mcheques_emitidos",
    "ccheques_depositados_rechazados",
    "mcheques_depositados_rechazados",
    "ccheques_emitidos_rechazados",
    "mcheques_emitidos_rechazados",
    "ccallcenter_transacciones",
    "chomebanking_transacciones",
    "ccajas_transacciones",
    "ccajas_consultas",
    "ccajas_depositos",
    "ccajas_extracciones",
    "ccajas_otras",
    "catm_trx",
    "matm",
    "catm_trx_other",
    "matm_other",
    "ctrx_quarter",
    "cmobile_app_trx",
    "Master_msaldototal",
    "Master_msaldopesos",
    "Master_msaldodolares",
    "Master_mconsumospesos",
    "Master_mconsumosdolares",
    "Master_mlimitecompra",
    "Master_madelantopesos",
    "Master_madelantodolares",
    "Master_mpagado",
    "Master_mpagospesos",
    "Master_mpagosdolares",
    "Master_mconsumototal",
    "Master_cconsumos",
    "Master_cadelantosefectivo",
    "Visa_msaldototal",
    "Visa_msaldopesos",
    "Visa_msaldodolares",
    "Visa_mconsumospesos",
    "Visa_mconsumosdolares",
    "Visa_mlimitecompra",
    "Visa_madelantopesos",
    "Visa_madelantodolares",
    "Visa_mpagado",
    "Visa_mpagospesos",
    "Visa_mpagosdolares",
    "Visa_mconsumototal",
    "Visa_cconsumos",
    "Visa_cadelantosefectivo"
]

campos_adicionales = [
    "tc_consumo_total",
    "tc_financiacionlimite_total",
    "tc_saldopesos_total",
    "tc_saldodolares_total",
    "tc_consumopesos_total",
    "tc_consumodolares_total",
    "tc_limitecompra_total",
    "tc_adelantopesos_total",
    "tc_adelantodolares_total",
    "tc_adelanto_total",
    "tc_pagado_total",
    "tc_pagadopesos_total",
    "tc_pagadodolares_total",
    "tc_saldototal_total",
    "tc_consumototal_total",
    "tc_cconsumos_total",
    "tc_morosidad_total",
    "m_promedio_plazofijo_total",
    "m_promedio_inversion_total",
    "m_promedio_caja_ahorro",
    "m_promedio_tarjeta_visa_consumo",
    "m_promedio_tarjeta_master_consumo",
    "m_promedio_prestamos_personales",
    "m_promedio_prestamos_prendarios",
    "m_promedio_prestamos_hipotecarios",
    "m_promedio_inversion2",
    "m_promedio_pagodeservicios",
    "m_promedio_pagomiscuentas",
    "m_promedio_cajeros_propios_descuentos",
    "m_promedio_tarjeta_visa_descuentos",
    "m_promedio_tarjeta_master_descuentos",
    "m_promedio_comisiones_mantenimiento",
    "m_promedio_comisiones_otras",
    "m_promedio_forex_buy",
    "m_promedio_forex_sell",
    "m_promedio_transferencias_recibidas",
    "m_promedio_transferencias_emitidas",
    "m_promedio_extraccion_autoservicio",
    "m_promedio_cheques_depositados",
    "m_promedio_cheques_emitidos",
    "m_promedio_cheques_depositados_rechazados",
    "m_promedio_cheques_emitidos_rechazados",
    "m_promedio_atm",
    "m_promedio_atm_other",
    "proporcion_financiacion_master_cubierto",
    "proporcion_limite_master_cubierto",
    "proporcion_financiacion_visa_cubierto",
    "proporcion_limite_visa_cubierto",
    "proporcion_financiacion_total_cubierto",
    "proporcion_limite_total_cubierto",
    "tc_proporcion_saldo_pesos",
    "tc_proporcion_saldo_dolares",
    "tc_proporcion_consumo_pesos",
    "tc_proporcion_consumo_dolares",
    "tc_proporcion_pago_pesos",
    "tc_proporcion_pago_dolares",
    "tc_proporcion_adelanto_pesos",
    "tc_proporcion_adelanto_dolares"
]

campos = campos_iniciales + campos_adicionales


In [17]:
nuevos_features =""
for campo in campos:
  nuevos_features += f"\n, regr_slope({campo}, cliente_antiguedad) over ventana_3 as {campo}_slope_3"
#print(nuevos_features)

In [ ]:
%%sql
create or replace table competencia_01_sumas_drifting as
select *
  {{nuevos_features}}
from competencia_01_sumas
window ventana_3 as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row)


In [19]:
# Calculo valor actual vs acumulado historico y vs promedio histórico.

In [20]:
nuevos_features = ""
for campo in campos:
    nuevos_features += f'''
                         , division_segura({campo}, avg(COALESCE({campo}, 0)) over ventana_hist) as ratio_actual_{campo}
                         , sum(COALESCE({campo}, 0)) over ventana_hist as sumcum_{campo}
                         , avg(COALESCE({campo}, 0)) over ventana_hist as avg_historico_{campo}
                     '''
#print(nuevos_features)

In [ ]:
%%sql
--create or replace table competencia_01_sumas_drifting as
select *
  {{nuevos_features}}
from competencia_01_sumas_drifting
window ventana_hist as (partition by numero_de_cliente order by foto_mes rows between 3 preceding and current row);

#### E. Exportación.

In [ ]:
%%sql
COPY competencia_01_sumas_drifting TO '{dataset_path}competencia_01_fe_z5.csv' (FORMAT CSV, HEADER TRUE);

#### F. Subo predicción a Kaggle con el ultimo modelo y estos nuevos datos.

In [1]:
base_path = 'C:/Users/c678456/Desktop/Ian/Maestría/Especializacion/2do_cuatrimestre/DMEyF/'
dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_01_fe.csv'

In [2]:
#1. Leo el modelo.
import pickle
filename = modelos_path + 'exp_206_rf_100_imputacion_media.sav'
model_rf = pickle.load(open(filename, 'rb'))

In [3]:
#2. Leo los datos.
import pandas as pd
data = pd.read_csv(dataset_path + dataset_file)

In [4]:
#3. Divido.
# Filtro los cuatro meses de entrenamiento.
mes_train = [202101,202102,202103,202104]
mes_test = 202106

# Separo entre Train y Test.
#i. Configuro Train.
X = data[data['foto_mes'].isin(mes_train)]
y = X['clase_ternaria']
X = X.drop(columns=['clase_ternaria'])
#ii. Configuro Test.
X_futuro = data[data['foto_mes'] == (mes_test)]
y_futuro = X_futuro['clase_ternaria']
X_futuro = X_futuro.drop(columns=['clase_ternaria'])

In [6]:
#a. Imputo con la Media.
from sklearn.impute import SimpleImputer
import numpy as np
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
Xi = imp_mean.fit_transform(X)
Xif = imp_mean.fit_transform(X_futuro)

In [ ]:
model_rf.fit(Xi,y)

In [ ]:
#4. Predigo Junio.
#i. Realizo la predicción de probabilidades usando el modelo entrenado.
predicciones = model_rf.predict_proba(X_futuro)
#ii. Encuentro el índice de la columna "BAJA+2".
indice_baja2 = model_rf.classes_.tolist().index("BAJA+2")
#iii. Agrego la columna de probabilidad de "BAJA+2" al DataFrame.
X_futuro['prob_baja2'] = predicciones[:, indice_baja2]
#iv. Solo envío estímulo a los registros con probabilidad de "BAJA+2" mayor a 1/40.
X_futuro['Predicted'] = (X_futuro['prob_baja2'] > 1/40).astype(int)
#v. Selecciono las columnas de interés.
resultados = X_futuro[["numero_de_cliente","Predicted"]].reset_index(drop=True) 
#vi. Exporto como archivo .csv.
nombre_archivo = "K105_001.csv"
ruta_archivo= "../../../exp/{}".format(nombre_archivo)
resultados.to_csv(ruta_archivo,index=False)

In [ ]:
#5. Envío a Kaggle.
#a. Importo librería.
from kaggle.api.kaggle_api_extended import KaggleApi
#b. Configura el API de Kaggle
api = KaggleApi()
api.authenticate()
#c. Defino los parámetros claves.
mensaje = f'Archivo {nombre_archivo}. Se predice Junio con modelo entrenado con Enero, Febrero, Marzo y Abril. Imputación con la Media. 100 Trials para búsqueda de hiperparámetros. Nuevas columnas con Feature Engineering.'
competencia = 'dm-ey-f-2024-primera'
#c. Subo la Submission.
api.competition_submit(file_name=ruta_archivo,message=mensaje,competition=competencia)